https://quilt-llava.github.io \
https://github.com/aldraus/quilt-llava \
https://huggingface.co/wisdomik/Quilt-Llava-v1.5-7b

In [1]:
import os
import pandas as pd
import json

#### Reading the Pathology Test Data

In [2]:
questions = pd.read_csv("../pathology_test_data/questions.csv")
questions

,image_name,question,original_responses
0,ck_PTGC_2x.jpeg,How can you best describe the low-power patter...,The lymph node shows a mixed follicular and in...
1,ck_PTGC_2x.jpeg,What are three main differential diagnostic co...,"Based on the low-power pattern, the primary co..."
2,ck_PTGC_5x.jpeg,What is the morphologic alteration being depic...,"The image shows a central enlarged, somewhat i..."
3,ck_PTGC_5x.jpeg,What is the expected immunoarchitecture of the...,This image shows an enlarged secondary follicl...
4,ck_serositis_4x.jpg,What is the most common source for the change ...,"The changes here show extensive serositis, wit..."
5,ck_serositis_4x.jpg,What is the specific anatomic region shown in ...,The right half shows the muscularis propria an...
6,ck_steatohepatitis_100x.jpg,If these histologic changes included conspicuo...,There is fatty liver disease and if concurrent...
7,ck_steatohepatitis_100x.jpg,What would the primary histologic feature to s...,Wilson's disease has many non-specific finding...
8,ck_steatohepatitis_200x.jpg,In an overweight adolescent with mildly increa...,"In the background of steatosis, there is incre..."


In [3]:
images_path = os.path.join(os.getcwd(), "../pathology_test_data/images")
images_list = os.listdir(images_path)
images_list

['ck_steatohepatitis_100x.jpg',
 'ck_PTGC_2x.jpeg',
 'ck_serositis_4x.jpg',
 'ck_steatohepatitis_200x.jpg',
 'ck_PTGC_5x.jpeg']

#### Creating a message structure

In [4]:
messages = []
for index, row in questions.iterrows():
    image_name = row["image_name"]
    question = row["question"]
    question_id = index
    
    message = {
                "question_id": question_id,
                "image": image_name,
                "text": question,
              }
    
    messages.append(message)

#### Creating a jsonl file with each line mentioning a specific question/image

In [5]:
os.makedirs("./temp_files", exist_ok=True)
file_name = "quilt_llava_vicuna_7b.jsonl"
file_path = os.path.join("./temp_files", file_name)
with open(file_path, "w") as f:
    for item in messages:
        f.write(json.dumps(item) + "\n")

#### Running the inference code from the command line

In [ ]:
!export PYTHONPATH=/data/mn27889/path-open-data/quilt-llava:$PYTHONPATH

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python llava/eval/model_vqa.py \
--model-path wisdomik/Quilt-Llava-v1.5-7b \
--image-folder /data/mn27889/path-open-data/pathology_test_data/images \
--question-file /data/mn27889/path-open-data/vlm_pathology_test_data_responses/temp_files/quilt_llava_vicuna_7b.jsonl \
--answers-file /data/mn27889/path-open-data/vlm_pathology_test_data_responses/temp_files/quilt_llava_vicuna_7b_resp.jsonl

#### Reading the Responses file

In [6]:
resp_file_name = "quilt_llava_vicuna_7b_resp.jsonl"
resp_file_path = os.path.join("./temp_files", resp_file_name)

indices = []
llava_responses = []
with open(resp_file_path, "r") as f:
    for line in f:
        json_data = json.loads(line)
        indices.append(json_data["question_id"])
        llava_responses.append(json_data["text"])

In [7]:
responses = pd.Series(llava_responses, index=indices)
responses

0    The low-power pattern of this lymph node revea...
1    1. Reactive lymphadenopathy: This is the most ...
2    The enlarged solitary lymph node from a young ...
3    In a normal lymph node, the mantle zone is typ...
4    The most common cause of distension in a neona...
5    The image on the right half of the image, whic...
6    If the histologic changes included conspicuous...
7    The primary histologic feature that would sugg...
8    The portal changes you're observing in this ov...
dtype: object

In [ ]:
os.makedirs("./responses", exist_ok=True)
questions['quilt-llava-vicuna-7b-response'] = responses
questions.to_csv("./responses/quilt_llava_vicuna_7b_responses.csv", index=False)